In [1]:
!pip install transformers datasets

In [13]:
from google.colab import files
uploaded = files.upload()

Saving Train.txt to Train (1).txt


In [16]:
from datasets import Dataset

# Load text manually
with open("Train.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Wrap in a Dataset format
train_dataset = Dataset.from_dict({"text": lines})

# Check
print(f"✅ Loaded dataset with {len(train_dataset)} samples")

✅ Loaded dataset with 3 samples


In [17]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Required to avoid padding error

In [18]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [20]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer
import os
os.environ["WANDB_DISABLED"] = "true"

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=2, training_loss=3.8005218505859375, metrics={'train_runtime': 38.129, 'train_samples_per_second': 0.079, 'train_steps_per_second': 0.052, 'total_flos': 195969024000.0, 'train_loss': 3.8005218505859375, 'epoch': 1.0})

In [21]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
result = generator("Once upon a time", max_length=50, num_return_sequences=1)
print(result[0]['generated_text'])

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Once upon a time there were two people in the city and a man in the city. It was the only man who did not believe it. In fact he went to the city of his father had said was his own."

As the city grew, so did the man, and he was finally able to find a way to get to the other side. The city of the city, the city of two men, was a place of a great, great great great great great great great great. And it was there that he found the man that had the city of two men, and they would be.
"

He was a man of great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great great gre